In [ ]:
import polars as pl

pl.Config.set_tbl_cols(50)
pl.Config.set_fmt_str_lengths(20)
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_tbl_rows(200)

In [ ]:
df = pl.read_parquet('games.parquet')
print(df.columns)

team1 = 'PIT'
team2 = 'BOS'

df = df.filter(((pl.col('homeTeam') == team1) | (pl.col('awayTeam') == team1)) & ((pl.col('homeTeam') == team2) | (pl.col('awayTeam') == team2)))
df = df.with_columns(
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)
df.group_by('winner').len()

In [ ]:
df = pl.read_parquet('games.parquet')

df = df.with_columns((pl.col('homePim') + pl.col('awayPim')).abs().alias('pim'))
df.sort(pl.col('pim'), descending=True)

In [ ]:


# Getting team stats
# team_stats = pl.read_parquet('team_stats.parquet')
# standings = pl.read_parquet('team_standings.parquet').unnest('teamName').rename({'default': 'teamFullName'})
# full_team_stats = team_stats.join(standings, on=('seasonId', 'teamFullName'))
# full_team_stats = full_team_stats.unnest('teamAbbrev').rename({'default': 'teamAbbrev'})

games = pl.read_parquet('games.parquet')
games = games.with_columns(
  pl.col('season').cast(pl.UInt64).alias('seasonId'),
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)

playoffs = games.filter(pl.col('gameType') == 3)
playoffs = playoffs.with_columns(
  pl.col('gameId').cast(pl.String).str.head(9).alias('series')
)
playoffs = playoffs.with_columns(
  (pl.col('winner') == pl.col('homeTeam').first()).cum_sum().over(pl.col('series')).shift(1).fill_null(0).alias('seriesHomeTeamWins'),
  (pl.col('winner') == pl.col('awayTeam').first()).cum_sum().over(pl.col('series')).shift(1).fill_null(0).alias('seriesAwayTeamWins'),
)

playoffs = playoffs.with_columns(
  pl.when(pl.col('homeTeam') == pl.col('homeTeam').first()).then(pl.col('seriesHomeTeamWins')).otherwise(pl.col('seriesAwayTeamWins')).alias('seriesHomeTeamWins'),
  pl.when(pl.col('awayTeam') == pl.col('awayTeam').first()).then(pl.col('seriesAwayTeamWins')).otherwise(pl.col('seriesHomeTeamWins')).alias('seriesAwayTeamWins')
)

playoffs = playoffs.with_columns(
  (pl.col('winner') == pl.col('homeTeam')).cast(pl.UInt64).alias('homeTeamWon'),
  (pl.col('winner') == pl.col('awayTeam')).cast(pl.UInt64).alias('awayTeamWon'),
)

playoffs = playoffs.select('gameId', 'gameType', 'seasonId', 'homeTeam', 'seriesHomeTeamWins', 'awayTeam', 'seriesAwayTeamWins', 'homeTeamWon', 'awayTeamWon')
playoffs